# Analysis of Lung LPS and tabula Senis data

In this Notebook the single cell RNAseq data from Zhang, L. et al. Single-cell transcriptomic profiling of lung endothelial cells identifies dynamic inflammatory and regenerative subpopulations. JCI Insight 7, (2022) is primarily analyzed.

The Motivation was to see how Aplnr expression reacts the the inflammation stimulus given and if senescence processes are initated in the endothelial cells. 



In [1]:
import scanpy as sc
import yaml
import os
import pandas as pd
import numpy as np
import decoupler as dc
import seaborn as sns


os.environ['HTTP_PROXY']="http://www-int.dkfz-heidelberg.de:80"
os.environ['HTTPS_PROXY']="http://www-int.dkfz-heidelberg.de:80"

In [2]:
!which python

/bin/python


In [3]:
if "snakemake" not in globals():
    with open("configs/lung_lps.yaml", "r") as stream:
        try:
            config=yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)

In [ ]:
if "snakemake" in globals():
    paths = snakemake.input["adata_paths"]
else:
    paths = [os.path.join(config["BASE_FP"], config["DATASET"], sample, "anndata", "adata_norm_processed.h5ad") for sample in config["samples"].keys()]

adata = list(map(sc.read_h5ad, paths))


## Full Dataset

We satrt off by analyzing these data by loading an adata object of all cells and adding the cluster annotaions by the authors (provided upon request)

Then we add the "immunec" and devEC populations based on the marker genes described in the paper above to assign the clusters to the cell tyes described.


The we plot the UMAPS

In [2]:
if "snakemake" in globals():
    adata_big = sc.read_h5ad(snakemake.input["adata_big"])
    adata_out_path = snakemake.output["adata_processed"]
else:
    adata_big=sc.read_h5ad("/omics/odcf/analysis/OE0228_projects/VascularAging/rna_sequencing/public_scrnaseq/lung_lps/merged/anndata/adata_merged.h5ad")

In [6]:
sample_names = ["basal", "Hour6","Day1", "Day2", "Day3", "Day7"]
labels = dict()
for s_name in sample_names:
    labels[s_name] = pd.read_csv(f"./data/lunglps/{s_name}_label.csv").set_index("Unnamed: 0")
    

In [7]:
labels["basal"] = labels["basal"].replace({1: "immunec", 2: "devEC", 3: "Aerocyte"})
labels["Hour6"] = labels["Hour6"].replace({1: "immunec", 2: "devEC", 3: "Aerocyte"})
labels["Day1"] = labels["Day1"].replace({1: "immunec", 2: "devEC", 3: "Aerocyte"})
labels["Day2"] = labels["Day2"].replace({1: "immunec", 2: "devEC", 4: "Aerocyte"})
labels["Day3"] = labels["Day3"].replace({1: "immunec", 3: "proEC", 2: "Aerocyte"})
labels["Day7"] = labels["Day7"].replace({1: "immunec", 2: "devEC", 4: "Aerocyte"})
labels_all = pd.concat(labels)
labels_all=labels_all.reset_index()
labels_all=labels_all.rename(columns={"level_0": "samples","Unnamed: 0": "cell_id"})


In [8]:
labels_all["cell_type"] = labels_all["Clusters"]

labels_all["cell_type"] = labels_all["cell_type"].replace({"devEC": "devEC/proEC", "proEC": "devEC/proEC"})

labels_all["cell_type"] = labels_all["cell_type"].where(labels_all["cell_type"].isin(["devEC/proEC", "Aerocyte", "immunec"])).replace({np.nan: "other"})


In [9]:
labels_all=labels_all.set_index("cell_id")

In [10]:
import scipy.sparse as sp

# Convert ndarray to csr_matrix



In [11]:
adata_big.obs["Clusters"] = labels_all["Clusters"]
adata_big.obs["cell_type"] = labels_all["cell_type"]
adata_big.obs["Clusters"][adata_big.obs["Clusters"].isnull()] = "missing"

sc.pp.filter_genes(adata_big, min_cells=1)
#sc.pp.highly_variable_genes(adata_big)


/tmp/ipykernel_30900/1600510565.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adata_big.obs["Clusters"][adata_big.obs["Clusters"].isnull()] = "missing"


MemoryError: Unable to allocate 4.69 GiB for an array with shape (32131, 19608) and data type int64

In [ ]:

sc.tl.pca(adata_big)
sc.pl.pca_variance_ratio(adata_big, log=False)

In [ ]:
sc.pp.neighbors(adata_big,n_pcs = 15)
sc.tl.umap(adata_big, )

In [ ]:
adata_big.obs["Clusters"] = adata_big.obs["Clusters"].astype("category")
sc.pl.umap(
    adata_big,
    color=["sample"]
)
sc.pl.umap(adata_big, color = "Clusters")
sc.pl.umap(adata_big, color = "cell_type")

These first wo UMAPS above show cells from all expreiments and the clusters annotated from in the second figure. 

Aerocytes cljuster separately to all other cells. Among the other ECs Hour6 both devEC and ImmuneEC populations cluster away -> Whereas in the other data these cell types seem to cluster in one large cluster in the middle. 

In [ ]:
sc.pl.umap(
    adata_big, 
    color=["Aplnr"],     
    vmin=0,
    vmax="p99",  # set vmax to the 99th percentile of the gene count instead of the maximum, to prevent outliers from making expression in other cells invisible. Note that this can cause problems for extremely lowly expressed genes.
    sort_order=False,  # do not plot highest expression on top, to not get a biased view of the mean expression among cells
    frameon=False,
    cmap="Reds")
        

## Compositional Analysis

Analyze the composition of the annoated EC cell types to track the changes. 


In [2]:

import pertpy as pt


In [3]:
sccoda_model = pt.tl.Sccoda()
sccoda_data = sccoda_model.load(
    adata_big,
    type="cell_level",
    cell_type_identifier="cell_type",
    sample_identifier="sample",
)
sccoda_data

NameError: name 'adata_big' is not defined

In [ ]:
pt.pl.coda.boxplots(
    sccoda_data,
    modality_key="coda",
    feature_name="sample",
    figsize=(12, 5),
    add_dots=False,
    args_swarmplot={"palette": ["red"]},
)


In [ ]:
pt.pl.coda.stacked_barplot(
    sccoda_data, modality_key="coda", feature_name="sample", figsize=(4, 2),
)

Aplnr Expression is primarily in the center cluster with some expression in the DevEC and immuneEC cluster in the center of the image. 

## Get Annotations

Functional annotations are added from MSIGDB and we also create a subdataset with signatures from FRIDMAN_SENSCENCE_UP/DN

In [ ]:
msigdb=pd.read_csv("resources/msig_mouse.csv.gz")
msig_small=msigdb[["genesymbol","geneset"]].drop_duplicates()


In [ ]:
senscence = msig_small[msig_small['geneset'].isin(['FRIDMAN_SENESCENCE_UP', 'FRIDMAN_SENESCENCE_DN'])]

In [ ]:
import pandas as pd

# Define the URL of the TSV file
url = "http://www.informatics.jax.org/downloads/reports/HMD_HumanPhenotype.rpt"

# Define the column names for the dataframe
col_names = ["hgene","hID", "mgene", "mID", "lcol", "ncol"]

# Use pandas to read the TSV file into a dataframe
mouse_human_homologs = pd.read_csv(url, sep="\t", header = None, index_col=None)
mouse_human_homologs.columns = col_names
# Print the first few rows of the dataframe


In [4]:
# Subset adata_object
adata_hvg = adata_big[:, adata_big.var["highly_variable"]]

NameError: name 'adata_big' is not defined

### EC.SENESCENCE.SIG 

THe gene list below was taken from Wu, Z., Uhl, B., Gires, O. & Reichel, C. A. A transcriptomic pan-cancer signature for survival prognostication and prediction of immunotherapy response based on endothelial senescence. J Biomed Sci 30, 1–19 (2023).
where a pan-cancer transriptome signature for EC senscence is descrbied. -> We checked its activitiy in this dataset to asses it versus other known senescence indicators. 

We translate the genes to the human homologs to start off. and then add it to our senescence signature of interest. 

In [ ]:
import pandas as pd

# List of genes as a Python Series
gene_series = [
    "FERMT2", "PLXNA2", "FLT1", "CAV2", "ICAM2", "GALNT18", "LAMA4", "SPARC", "PCDH12",
    "PLEKHG1", "MYCT1", "EFNB2", "CD93", "RHOJ", "KDR", "PODXL", "DLL4", "DOCK6", "PLVAP",
    "TMEM204", "NES", "COL4A2", "HECW2", "DUSP6", "ACVRL1", "PTPRG", "ESAM", "PRSS23", "GJA1",
    "AFAP1L1", "STC1", "COX7A1", "ITGA5", "BCL6B", "IGFBP7", "TM4SF18", "DLC1", "JCAD", "CYYR1",
    "SYNPO", "MMRN2", "CD34", "FZD4", "A2M", "CAVIN1", "CDH5", "IL3RA", "BCAM", "COL4A1",
    "S100A16", "TCF4", "TGM2", "BMPR2", "SCARF1", "ECE1", "PLK2", "RHOC", "SERPINH1", "INSR",
    "IPO11", "MAGI1", "NID1", "MECOM", "UACA", "TUBB6", "LMO2", "NECTIN2", "GRB10", "LAMA5",
    "LUZP1", "MAST4", "DYSF", "PNP", "NRP1", "CAVIN3", "LRRC8A", "EFNA1", "NFIA", "EHD4",
    "TNFAIP1", "PLXND1", "LAMB1", "RGS3", "ZEB1", "TRIOBP", "FSCN1", "YES1", "JAG1", "PEA15",
    "RAB13", "PHACTR2", "LAMC1", "VWA1", "PPIC", "SLC44A2", "PLEKHA1", "TPM4", "GNAI2", "MGLL",
    "UTRN", "CAPNS1"
]

# Print the Series 
print(gene_series)


In [ ]:
mouse_genes = mouse_human_homologs[mouse_human_homologs["hgene"].isin(gene_series)]
mouse_genes

In [ ]:
EC_sig = pd.DataFrame({"genesymbol": mouse_genes.mgene, "geneset": "ec_Senescence_sig"})

In [ ]:
import pandas as pd
# GEt Quiescence Signatures read from "../multicondition-deseq2-enrichment/data/senescence_genesets.txt"

filepath = "../multicondition-deseq2-enrichment/data/senescence_genesets.txt"
df = pd.read_csv(filepath, sep='\t')
# change df colnames to "genesymbol" and "geneset"
df.columns = ["geneset", "genesymbol"]
df



In [ ]:
full_sen =pd.concat([senscence, EC_sig,df])

In [ ]:
hallmarks = msigdb[msigdb['collection']=='hallmark']

# Remove duplicated entries
hallmarks = hallmarks[~hallmarks.duplicated(['geneset', 'genesymbol'])]
hallmarks

In [ ]:
#adata_big.X = adata_big.layers["scran_normalization"]

In [ ]:
#adata_big.layers

In [ ]:
dc.run_ora(
    mat=adata_hvg,
    net=pd.concat([hallmarks, full_sen]),
    source='geneset',
    target='genesymbol',
    verbose=True,
    use_raw=False
)

In [ ]:
dc.run_aucell(
    mat=adata_hvg,
    net=pd.concat([hallmarks, full_sen]),
    source='geneset',
    target='genesymbol',
    verbose=True,
    use_raw=False
)

In [ ]:
labels_all.index = labels_all.index.to_flat_index()

In [ ]:
adata_big
sc.pl.violin(adata_big, keys = ["Aplnr", "Junb"], groupby = "sample", order=sample_names)
adata_big.obs["Clusters"]

In [ ]:
sc.pl.dotplot(adata_big, var_names = ["Aplnr", "Apln"], groupby = "sample", categories_order = sample_names)

In [ ]:
adata_big.obs["sample_cluster"] = adata_big.obs["sample"].astype("str") + "_" + adata_big.obs["cell_type"].astype("str")

sc.pl.dotplot(adata_big, var_names = ["Aplnr", "Apln"], groupby = "cell_type")

In [ ]:
sc.pl.dotplot(adata_big, var_names = ["Aplnr", "Apln"], groupby = "sample_cluster")

Here we clearly see that Aplnr Expression is present at the start but after treatment with LPS drops to almost 0 by day1. However by Day7 -> Aplnr Expression increases and overshoots the expression from the start of the experiment

Lastly, we have som extra plots below with alternative color schemes and looking at Aplnr Expression across the different cell types. 

In [ ]:
sc.pl.umap(
    adata_big,
    color=["Prox1","Nr2f2", "Aplnr"],     
    vmin=0,
    vmax="p99",  # set vmax to the 99th percentile of the gene count instead of the maximum, to prevent outliers from making expression in other cells invisible. Note that this can cause problems for extremely lowly expressed genes.
    sort_order=False,  # do not plot highest expression on top, to not get a biased view of the mean expression among cells
    frameon=False,
    cmap="Reds"
)


In [ ]:
adata_big

In [ ]:
sc.pl.umap(
    adata_big,
    color=["Aplnr"],
    vmax="p99"
)
sc.pl.violin(
    adata_big,
    keys= ["Aplnr"],
    groupby="cell_type"
)

## Enrichments for adata big Lung lps

To get a better understanding of the data, we performed an over representation analysis (using decoupler-py) against the HALLMARK gene sets from Msigdb, FRIDMAN_SENESCENCE_UP/DN and the EC.SENESCENCE.SIG signature.




### Enrichment using ORA tests


In [ ]:
adata_hvg.X = np.nan_to_num(adata_hvg.X)

In [ ]:
acts = dc.get_acts(adata_hvg, obsm_key='ora_estimate')

# We need to remove inf and set them to the maximum value observed
acts_v = acts.X.ravel()
max_e = np.nanmax(acts_v[np.isfinite(acts_v)])
acts.X[~np.isfinite(acts.X)] = max_e

acts



acts.obsm["ora_estimate"].columns

In [ ]:
gs_of_interest = ['HALLMARK_DNA_REPAIR', 'HALLMARK_INTERFERON_ALPHA_RESPONSE', 
                        'FRIDMAN_SENESCENCE_DN', 'FRIDMAN_SENESCENCE_UP', 'ec_Senescence_sig']
sc.pl.umap(acts, color=gs_of_interest, cmap='RdBu_r',vmax = "p99",ncols=3)
sc.pl.umap(acts, color="sample")

sc.pl.violin(acts, keys=gs_of_interest, groupby='sample', rotation=90, multi_panel=True,order =["basal", "Hour6", "Day1", "Day2", "Day3", "Day7"] )
sc.pl.violin(acts, keys=gs_of_interest, groupby='cell_type', rotation=90, multi_panel=True )


Here we the activity of some selected gene sets in these data. 
. First we have the per cell Enrichment between six gene sets. After Stimulus at Hour6 Intereron Response Increases due ot the DNA Damage. DNA Repair is active in a subset of proECs at day3. 

The Senescene signatures show now clear signs in the UMAP, however in the Violin plots we see some evidence that the EC Senescence signature falls a bit during inflammation and is increased by day7. HOwever the picture is not very clear here. 

Next we rank the group sources by testing if the change between one samples vs all other samples is significant in some of the gene sets. 

We than take the top five per group and produce the matrix plot below. 

In [ ]:
df = dc.rank_sources_groups(acts, groupby='sample', reference='rest', method='t-test_overestim_var')
df

df_basal = dc.rank_sources_groups(acts, groupby='sample', reference='basal', method='t-test_overestim_var')



In [ ]:
df_basal

In [ ]:
n_markers = 5
source_markers = df.groupby('group').head(n_markers).groupby('group')['names'].apply(lambda x: list(x)).to_dict()
source_markers =  {k: source_markers[k] for k in ["basal", "Hour6", "Day1", "Day2", "Day3", "Day7"]}

n_markers = 5
source_markers_basal = df_basal.groupby('group').head(n_markers).groupby('group')['names'].apply(lambda x: list(x)).to_dict()
source_markers_basal =  {k: source_markers_basal[k] for k in ["Hour6", "Day1", "Day2", "Day3", "Day7"]}




In [ ]:
sc.pl.matrixplot(acts, source_markers, 'sample', dendrogram=False, title="vs_all",
                 colorbar_title='Z-scaled scores',standard_scale='var', cmap='RdBu_r',categories_order = ["basal", "Hour6", "Day1", "Day2", "Day3", "Day7"])

sc.pl.matrixplot(acts, source_markers_basal, 'sample', dendrogram=False, title="vs basal",
                 colorbar_title='Z-scaled scores',standard_scale='var', cmap='RdBu_r',categories_order = ["basal", "Hour6", "Day1", "Day2", "Day3", "Day7"])

Here the Activitiy of the top 5 enriched processes per samples is assesed across all samples. -> 
Two versions of this analysis were created. IN the first one we compare the enrichments as SAMPLe vs ALL other smaples and in the second analysis we set "basal as the baseline comparison. 
Results are similar. 

We see that the EC Senescence_sig is enriched on  DAY7. Inflammatory pathways are up from Hour6- day2 and On day3 repair processes are upregulated. 


### Enrichments using AUcell

Here we use an alternative strategy for enrichment analysis. Since we have no weights assigned to genes we stuck to another method that doesn expect weighted gene sets (AUcell)

Plots are corresponding to the OR analysis. 


In [ ]:
acts = dc.get_acts(adata_hvg, obsm_key='aucell_estimate')

# We need to remove inf and set them to the maximum value observed
acts_v = acts.X.ravel()
max_e = np.nanmax(acts_v[np.isfinite(acts_v)])
acts.X[~np.isfinite(acts.X)] = max_e

acts



acts.obsm["aucell_estimate"].columns

In [ ]:
gs_of_interest = ['HALLMARK_DNA_REPAIR', 'HALLMARK_INTERFERON_ALPHA_RESPONSE', 
                        'FRIDMAN_SENESCENCE_DN', 'FRIDMAN_SENESCENCE_UP', 'ec_Senescence_sig']
sc.pl.umap(acts, color=gs_of_interest, cmap='RdBu_r',vmax = "p99",ncols=3)
sc.pl.umap(acts, color="sample")

sc.pl.violin(acts, keys=gs_of_interest, groupby='sample', rotation=90, multi_panel=True,order =["basal", "Hour6", "Day1", "Day2", "Day3", "Day7"] )


In [ ]:
import seaborn as sns
from scipy.stats import pearsonr
import pandas as pd

selected_indices = adata_hvg.obs.index[adata_big.obs['cell_type'].isin(['devEC/proEC', 'immunec'])]

aucell_res = acts.obsm["aucell_estimate"].loc[selected_indices]
Aplnr_expr = pd.Series(adata_big[selected_indices, "Aplnr"].X.todense().A1)

correlations = []
p_values = []
for col in aucell_res:
    corr, p_value = pearsonr(Aplnr_expr, aucell_res[col])
    correlations.append(corr)
    p_values.append(p_value)

# Create a dataframe with the correlations, p-values, and index of aucell_res
df = pd.DataFrame({'correlation': correlations, 'p-value': p_values}, index=aucell_res.columns)

# Plot the correlations as a heatmap using seaborn
sns.heatmap(df, cmap='coolwarm')

In [ ]:
# Create a dataframe with the correlations and index of aucell_res
#df = pd.DataFrame({'correlation': correlations}, index=aucell_res.columns)
df = df.sort_values(by='correlation', ascending=False)
df

In [ ]:
# Plot the correlations as a heatmap using seaborn
sns.heatmap(df, cmap='coolwarm')

In [ ]:
df = dc.rank_sources_groups(acts, groupby='sample', reference='rest', method='t-test_overestim_var')
df
df_basal = dc.rank_sources_groups(acts, groupby='sample', reference='basal', method='t-test_overestim_var')
df_basal

In [ ]:
n_markers = 5
source_markers = df.groupby('group').head(n_markers).groupby('group')['names'].apply(lambda x: list(x)).to_dict()
source_markers =  {k: source_markers[k] for k in ["basal", "Hour6", "Day1", "Day2", "Day3", "Day7"]}

source_markers_basal = df_basal.groupby('group').head(n_markers).groupby('group')['names'].apply(lambda x: list(x)).to_dict()
source_markers_basal =  {k: source_markers_basal[k] for k in ["Hour6", "Day1", "Day2", "Day3", "Day7"]}



In [ ]:
sc.pl.matrixplot(acts, source_markers, 'sample', dendrogram=False,standard_scale="var", title= "vs all",
                 colorbar_title='Z-scaled scores', cmap='RdBu_r', categories_order = ["basal", "Hour6", "Day1", "Day2", "Day3", "Day7"])

sc.pl.matrixplot(acts, source_markers_basal, 'sample', dendrogram=False,standard_scale="var", title = "vs basal",
                 colorbar_title='Z-scaled scores', cmap='RdBu_r', categories_order = ["basal", "Hour6", "Day1", "Day2", "Day3", "Day7"])

In [ ]:
adata_big.obs["Clusters"] = adata_big.obs["Clusters"].astype("string")

for col in adata_big.obs.columns:
    #check dtype of column
    if adata_big.obs[col].dtypes in ("string", "category"):
        # if dtype is string convert to category
        adata_big.obs[col] = adata_big.obs[col].astype("object")

adata_big.write_h5ad(adata_out_path)

Enrichments usinsg AUCell are very similar to ORA -> So we refer to the ORA results. 